In [2]:
import numpy as np
import pandas as pd
from is_slum2 import get_slum_val

In [4]:
%%capture
from tqdm.notebook import tqdm
tqdm().pandas()

In [5]:
rio = pd.read_csv('../data/rio/cleaned_2.csv')
mumbai1 = pd.read_csv('../data/mumbai/cleaned_housing.com.csv')
mumbai2 = pd.read_csv('../data/mumbai/cleaned_99acres.com.csv')
mumbai = pd.concat([mumbai1[['lat', 'lng', 'price', 'coord']],
                    mumbai2[['lat', 'lng', 'price_per_month', 'coord']
                    ].rename(columns={'price_per_month':'price'})]).rename(
                             columns={'lat':'latitude','lng':'longitude'})
hyderabad = pd.read_csv('../data/hyderabad/cleaned.csv')
chennai = pd.read_csv('../data/chennai/cleaned.csv')

In [6]:
def Prepare(df):
    df['coord'] = list(zip(df['latitude'], df['longitude']))
    df.drop(columns=['latitude', 'longitude'], inplace=True)
    print(df.duplicated().sum(), 'duplicates dropped.')
    df.drop_duplicates(inplace=True)
    return df

In [7]:
rio = Prepare(rio)

137 duplicates dropped.


In [8]:
mumbai = Prepare(mumbai)

7409 duplicates dropped.


In [9]:
hyderabad = Prepare(hyderabad)

33 duplicates dropped.


In [26]:
def convert_lak_price(x):
    if x < 1000: return 100000*x
    else: return x
mumbai['price'] = mumbai['price'].apply(convert_lak_price)

### Classifying

In [28]:
def get_classes(df, city):
    class_map = dict()
    uniques = list(set(df['coord']))
    for coord in tqdm(uniques):
        try: class_map[coord] = get_slum_val(city, (coord[0], coord[1]))
        except: class_map[coord] = np.nan
    print(len(uniques), 'unique locations classified. mapping back to dataset...')
    df['class'] = df.coord.progress_apply(lambda x: class_map[x])
    print('COMPLETE. saving to CSV...')
    df.to_csv(f'classified_{city}.csv', index=False)
    print('saved. value counts:', df['class'].value_counts(normalize=True).values)

In [29]:
get_classes(rio, 'rio')


1617 unique locations classified. mapping back to dataset...



COMPLETE. saving to CSV...
saved. value counts: [0.95574163 0.04425837]


In [ ]:
get_classes(mumbai, 'mumbai')

In [ ]:
get_classes(hyderabad, 'hyderabad')